In [ ]:
#import the related modules 
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np
import os
import pickle
import random
import operator
import librosa
import librosa.display
import IPython.display as ipd 
import math
import seaborn as sns
from sklearn.neighbors import kneighbors_graph
import matplotlib.pyplot as plt


In [ ]:
#function to get the distance between feature vectors and find neighbors

def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range(len(trainingSet)):
        dist = distance(trainingSet[x], instance, k) + distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))

    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    
    return neighbors

In [ ]:
#identify the class of instance 
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)

    return sorter[0][0]

In [ ]:
#function to evaluate the model 
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    
    return (1.0 * correct) / len(testSet)

In [ ]:
#directory that holds the .wav files 
# add the path of directory of the folder containing the dataset
directory = "C:/Users/HP/Desktop/music/genres_original/"


In [ ]:
#binary file to collect all the features extracted usingin mfcc 

f = open("my.dat", 'wb')

i = 0

for folder in os.listdir(directory):
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+folder):        
        try:
            (rate, sig) = wav.read(directory+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print('Got an exception: ', e, ' in folder: ', folder, ' filename: ', file)        

f.close()


In [ ]:
## features music genre distribution across the dataset
plt.plot(mfcc_feat)
plt.xlabel("Rate")
plt.ylabel("Sig")
plt.show()

In [ ]:
## relation between the features in this case genre of each music type
## shows that no any features are tightly dependet upon each other
plt.plot(covariance)
plt.show()

In [ ]:
#split the dataset into training and testing set 

dataset = []

def loadDataset(filename, split, trSet, teSet):
    with open('my.dat', 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])
trainingSet = []
testSet = []
loadDataset('my.dat', 0.80, trainingSet, testSet)

In [ ]:
#function to perform actual distance calculations between features

def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance



In [ ]:

# making predictions using KNN
leng = len(testSet)
predictions = []
for x in range(leng):
    predictions.append(nearestClass(getNeighbors(trainingSet, testSet[x], 7)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

In [ ]:
x_axis=[10]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as mtick

learning_rate=0.1
accuracy=[float(accuracy1)]
accuracy_threshold=int(accuracy[0]*learning_rate*70)
data=[]
for accuracy_range in range(len(accuracy),accuracy_threshold):
    data.append(accuracy_range)
data.append(accuracy[0]*100)
perc = np.linspace(0,100,len(data))

fig = plt.figure(1, (7,4))
ax = fig.add_subplot(1,1,1)

ax.plot(perc, data)

fmt = '%.0f%%' # Format you want the ticks, e.g. '40%'
xticks = mtick.FormatStrFormatter(fmt)
ax.xaxis.set_major_formatter(xticks)

plt.xlabel(" Testing Accuracy")
plt.ylabel("Value of K for KNN")
plt.show()
## improvement on accuracy after each epoches (after finding best neighbor values i.e. 3)

In [ ]:
#testing the code 
test_dir = "C:/Users/HP/Desktop/music/test/" #add the path to the test folder on your device 
test_file = test_dir + "test2.wav" #add your test file 

In [ ]:
#play audio file 
ipd.Audio(test_file)

In [ ]:
#extracting features 
(rate, sig) = wav.read(test_file)
mfcc_feat = mfcc(sig, rate, winlen = 0.020, appendEnergy = False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix, covariance, i)


In [ ]:

from collections import defaultdict
results = defaultdict(int)

directory = "C:/Users/HP/Desktop/music/genres_original/"  #add the path of directory of the folder containing the dataset 


i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [ ]:
music_type=[]
music_value=[]
colors=['red','green','orange','purple','grey','black','violet','cyan','pink']
for key,value in results.items():
    music_type.append(value)
    music_value.append(key)

from matplotlib.pyplot import figure

figure(figsize=(8, 6), dpi=80)
bar=plt.bar(music_type,music_value)
for index,each in enumerate(colors):
    bar[index].set_color(each)

 ## music genre list with unique id from 0 to 10 [ordinal categories] the higher ranking of the number from 0 to 10 has nothing to do with any observation
plt.xlabel("Genres")
plt.show()

In [ ]:
pred = nearestClass(getNeighbors(dataset, feature, 7))
print("The genre is " + results[pred])